In [4]:
import torch
import torch.nn as nn

In [15]:
class MultiKernelBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernels):
        super(MultiKernelBlock, self).__init__()
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
                nn.AvgPool2d(kernel_size=(5,1), stride=(5,1))
            ) for kernel in kernels
        ])
        nn.Dropout(p=0.2)

    def forward(self, x):
        outputs = [conv(x) for conv in self.convs]
        return torch.cat(outputs, 1)


In [16]:
class SeparableConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, kernel_size, bias=False):
        super(SeparableConv2d, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels, kernel_size=kernel_size, 
                                groups=in_channels, bias=bias, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)  # Added BatchNorm after depthwise convolution
        self.pointwise = nn.Conv2d(in_channels, out_channels, 
                                kernel_size=1, bias=bias)
        self.bn2 = nn.BatchNorm2d(out_channels)  # Added BatchNorm after pointwise convolution
        self.relu = nn.ReLU()  # Added ReLU activation

    def forward(self, x):
        out = self.depthwise(x)
        out = self.bn1(out)  # Apply BatchNorm after depthwise convolution
        out = self.relu(out)  # Apply ReLU activation
        out = self.pointwise(out)
        out = self.bn2(out)  # Apply BatchNorm after pointwise convolution
        out = self.relu(out)  # Apply ReLU activation
        return out

In [17]:
class TemporalSEBlock(nn.Module):
    def __init__(self, in_channels):
        super(TemporalSEBlock, self).__init__()
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  # Changed to global pooling
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // 16, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // 16, in_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.global_pool(x).view(b, c)  # Changed to global pooling
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)


In [ ]:
class SpatialSE(nn.Module):
    def __init__(self, in_channels):
        super(SpatialSE, self).__init__()
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  # Changed to global pooling
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels // 16, bias=False),  # Two dense layers to reduce the channel complexity
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // 16, in_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.global_pool(x).view(b, c)  # Changed to global pooling
        y = self.excitation(y).view(b, c, 1, 1)
        return x * y.expand_as(x)  # After performing these convolutions, the extracted features are generated and transmitted to the classification layer


In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.multi_kernel_block = MultiKernelBlock(in_channels=3, out_channels=64, kernels=[5, 10, 20, 40])
        self.temp_SE_block = TemporalSEBlock(64)  # Corrected the spelling here and added the required argument
        self.separable_conv2d = SeparableConv2d(in_channels=64, out_channels=128, kernel_size=3)  # Added this line to use the SeparableConv2d block
        self.avg_pool = AvgPool2d(10*1, stride=10*1)
        self.dropout = dropout(p=.2)
    def forward(self, x):
        x = self.multi_kernel_block(x)
        x = self.temp_SE_block(x)  # Added this line to use the TemporalSEBlock
        x = self.separable_conv2d(x)  # Added this line to use the SeparableConv2d block
        return x